In [2]:
from os.path import join
import jpype
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, isJVMStarted
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [3]:
from os.path import join
import jpype
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, isJVMStarted
import pandas as pd
import numpy as np
import re

class Text_Cleaner:
    
    def __init__(self,):
        self.zemberek() 
              
    def text_cleaner(self,ds): #data_Seri - data.content
        # kullanmadan önce dropna yap
        array = []
        ds = ds.apply(lambda x: self.norm_doc(str(x)))
        ds = ds.str.replace("I","ı").str.replace("İ","i").str.lower().str.replace("[^\w\s]"," ")
#         ds = ds.apply(lambda x: re.sub(r'\b\w\b', "", x))#zembereksiz için
#         return ds#zembereksiz için
        for i in tqdm(ds): #kelime --> zemberek
            a =  ""
            for t in i.split():
                try:  
                    results: WordAnalysis = self.morphology.analyze(JString(t))
                        
                    if "Neg" in str(list(results)[0].formatLong()):
                        wrd = str(list(results)[0].getLemmas()[1])
                    else:
                        wrd = str(list(results)[0].getStems()[0])
                    a +=  wrd + " " 

                except:
                    continue
            a = re.sub(r'\b\w\b', "", a) # tekharfler silindi
            array.append(a)
            
        return array
        
        
        
    def norm_doc(self,sentence): 
        sentence = re.sub("'[A-Za-z]+", "", sentence) #tırnak eki silindi
        
        return re.sub("\d+", "", sentence) #sayılar silindi
    
    
    def zemberek(self,):
        ZEMBEREK_PATH: str = join('bin', 'zemberek-full.jar')
        if not jpype.isJVMStarted():   
            startJVM( getDefaultJVMPath(),'-ea',f'-Djava.class.path={ZEMBEREK_PATH}',convertStrings=False)
        TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology') # burada java'da import etmişsin gibi düşün
        WordAnalysis: JClass = JClass('zemberek.morphology.analysis.WordAnalysis') # aynı şekilde
        self.morphology: TurkishMorphology = TurkishMorphology.createWithDefaults() # burada da o import'u kullanıyorsun
            
    def zemberek_shutdown(self,):
        shutdownJVM()
    

In [4]:
data = pd.read_csv("../Data/Labelled_temiz_banka_news.csv")


In [5]:
tc = Text_Cleaner()

In [7]:
data.content = tc.text_cleaner(data.content)

100%|████████████████████████████████████████████████████████████████████████████| 10589/10589 [04:22<00:00, 40.29it/s]


In [11]:
data.to_csv("../Data/Zemberekli_temiz_banka_Labelled_news.csv")

In [10]:
data.content.head(10).tolist()

['bıst endeks dü art puan kapat böylece gün düş sonra gün üst üs yüksel ol banka dinlen sanayi endeks yüksel borsa istanbul en kıymet şirket ara piyasa değer göre yap bir süre de bank lider söz konu mart kapan itibari de bank milyar tl milyar ile ilk sıra ye al koç holding milyar tl ile ikinci sıra üçüncü ve dördüncü biraz geri gel garanti banka milyar tl ile üçüncü tüpraş milyar tl ile dördüncü sıra milyar tl ile akbank milyar tl ile ereğli demir çelik ve milyar tl ile turkcell milyar aş diğer şirket son dakika bilgi ver genel müdür terzioğlu ayrıl yer için bugün yap çok değil ay önce ocak denizbank milyar tl ile yine lider koç holding milyar tl garanti banka milyar tl akbank milyar tl ile milyar aş şirket son döne sürpriz hisse ol işbank ve tskb ortaklığ iş finansal  gün milyar tl piyasa değer ile sıra iken dün milyar tl baraj geç sıra yüksel ingiltere demokrasi şov devam ed kendi kader kimse de göre değil ortak akıl çaba takdir ed gün sür fark politika seçim ingiliz parlamento yenid